In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==4.31.0

In [ ]:
!gdown 1uMZwc1FeFmXmgyDwhKFietC-ammrT_NG
df_cleaned = pd.read_csv("preprocessed.csv")

In [ ]:
from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(df_cleaned, test_size = 0.2, random_state = 42, stratify = df_cleaned.sentiment)

In [ ]:
data_test.to_csv("data_test.csv", index=False)
data_train.to_csv("data_train.csv", index=False)

In [ ]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a 

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/bertt/bert-tokenizer')
bert.save_pretrained('/content/drive/MyDrive/bertt/bert-model')

In [ ]:
import shutil
shutil.make_archive('bert-tokenizer', 'zip', 'bert-tokenizer')

'/content/bert-tokenizer.zip'

In [ ]:
shutil.make_archive('bert-model','zip','bert-model')

'/content/bert-model.zip'

In [ ]:
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
tokenizer('hello i am depressed right now and i am so want to die')

{'input_ids': [101, 7592, 1045, 2572, 14777, 2157, 2085, 1998, 1045, 2572, 2061, 2215, 2000, 3280, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
x_train = tokenizer(
    text=data_train.content.tolist(),
    add_special_tokens=True,
    max_length=53,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=data_test.content.tolist(),
    add_special_tokens=True,
    max_length=53,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
x_test['input_ids']

<tf.Tensor: shape=(8638, 53), dtype=int32, numpy=
array([[  101,  2008,  2003, ...,  9906,  3228,   102],
       [  101,  5191, 20334, ...,  7216,  4224,   102],
       [  101,  2215,  2619, ...,     0,     0,     0],
       ...,
       [  101,  2228,  3784, ...,     0,     0,     0],
       [  101,  2009,  4332, ...,     0,     0,     0],
       [  101,  2514,  2066, ...,  2894,  2123,   102]], dtype=int32)>

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
max_len = 53
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(5,activation = 'sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 53)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 53)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 53, 76                                     

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train.label),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.label)
    ),
  epochs=1,
    batch_size=36
)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


  1/960 [..............................] - ETA: 3:33:50 - loss: 2.1557 - balanced_accuracy: 0.2222

  2/960 [..............................] - ETA: 26:25 - loss: 1.9650 - balanced_accuracy: 0.1806  

  3/960 [..............................] - ETA: 24:37 - loss: 1.8520 - balanced_accuracy: 0.2037

  4/960 [..............................] - ETA: 23:13 - loss: 1.7874 - balanced_accuracy: 0.2153

...

956/960 [============================>.] - ETA: 3s - loss: 0.7949 - balanced_accuracy: 0.6724

957/960 [============================>.] - ETA: 2s - loss: 0.7947 - balanced_accuracy: 0.6725

958/960 [============================>.] - ETA: 1s - loss: 0.7947 - balanced_accuracy: 0.6725

959/960 [============================>.] - ETA: 0s - loss: 0.7944 - balanced_accuracy: 0.6727

960/960 [==============================] - 998s 1s/step - loss: 0.7946 - balanced_accuracy: 0.6726 - val_loss: 0.7291 - val_balanced_accuracy: 0.7055


In [ ]:
model.save_weights('/content/drive/MyDrive/bertt/sentiment_weights.h5')

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

270/270 [==============================] - 67s 249ms/step


In [ ]:
predicted_raw[0]

array([0.9876601 , 0.9493194 , 0.03534099, 0.5477407 , 0.03022037],
      dtype=float32)

In [ ]:
y_predicted = np.argmax(predicted_raw, axis = 1)

In [ ]:
data_test.label

202      0
29520    3
31696    3
39969    4
7979     0
        ..
12805    1
38396    4
30584    3
24507    2
31060    3
Name: label, Length: 8638, dtype: int64

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(data_test.label, y_predicted))

              precision    recall  f1-score   support

           0       0.78      0.64      0.70      1728
           1       0.72      0.58      0.65      1727
           2       0.68      0.75      0.71      1728
           3       0.76      0.90      0.82      1728
           4       0.61      0.65      0.63      1727

    accuracy                           0.71      8638
   macro avg       0.71      0.71      0.70      8638
weighted avg       0.71      0.71      0.70      8638



In [ ]:
# copy it there
!cp sentiment_weights.h5 /content/drive/MyDrive/bertt
!cp bert-model.zip /content/drive/MyDrive/bertt
!cp bert-tokenizer.zip /content/drive/MyDrive/bertt

cp: cannot stat 'sentiment_weights.h5': No such file or directory


In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/bertt2/bert-tokenizer')
bert.save_pretrained('/content/drive/MyDrive/bertt2/bert-model')

In [ ]:
train_history2 = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train.label),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.label)
    ),
  epochs=3,
    initial_epoch=1,
    batch_size=36
)

Epoch 2/3


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


  1/960 [..............................] - ETA: 19:29 - loss: 0.5585 - balanced_accuracy: 0.7778

  2/960 [..............................] - ETA: 25:15 - loss: 0.6448 - balanced_accuracy: 0.6944

  3/960 [..............................] - ETA: 31:36 - loss: 0.6330 - balanced_accuracy: 0.7222

...

956/960 [============================>.] - ETA: 3s - loss: 0.6396 - balanced_accuracy: 0.7462

957/960 [============================>.] - ETA: 2s - loss: 0.6401 - balanced_accuracy: 0.7461

958/960 [============================>.] - ETA: 1s - loss: 0.6399 - balanced_accuracy: 0.7462

959/960 [============================>.] - ETA: 0s - loss: 0.6399 - balanced_accuracy: 0.7462

960/960 [==============================] - 991s 1s/step - loss: 0.6398 - balanced_accuracy: 0.7463 - val_loss: 0.7073 - val_balanced_accuracy: 0.7117


In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/bertt4/bert-tokenizer')
model.save('/content/drive/MyDrive/bertt4/bert-model')
model.save_weights('/content/drive/MyDrive/bertt4/sentiment_weights.h5')

In [ ]:
texts = input(str('input the text'))

x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=53,
    truncation=True,
    padding='max_length',
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
validation

input the textalone


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 261ms/step


array([[32.273636, 58.46719 , 51.089634, 83.55497 , 80.033485]],
      dtype=float32)

In [ ]:
label_dict = dict(zip(df.sentiment, df.label))
sentiment_list = sorted(label_dict.keys(), key=lambda x:x.lower())

for key , value in zip(sentiment_list,validation[0]):
    print(key,value)

anxiety 32.273636
depression 58.46719
happiness 51.089634
lonely 83.55497
neutral 80.033485


In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 53)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 53)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 53, 76                                     